<hr />

### NoteBookの見方
コード以外の情報 
<span >白/黒での記載は速習Symbol</span>  
<span style="color:red">赤色での記載は補足情報</span>  
<span >コード内で🌟マークがある場所は自分の情報に書き換えが必要</span>

<hr />

<span style="color:red">

# 環境構築
</span>
<span style="color:red">

## 1.Symbol SDKの読み込み </span>

In [1]:
(script = document.createElement("script")).src = "https://xembook.github.io/nem2-browserify/symbol-sdk-pack-2.0.3.js";
document.getElementsByTagName("head")[0].appendChild(script);

<span style="color:red">

## 2.Symbol用の共通設定 </span>

In [2]:
NODE = 'https://sym-test-03.opening-line.jp:3001';
sym = require("/node_modules/symbol-sdk");
repo = new sym.RepositoryFactoryHttp(NODE);
txRepo = repo.createTransactionRepository();
mosaicRepo = repo.createMosaicRepository();
accountRepo = repo.createAccountRepository();
(async () => {
  networkType = await repo.getNetworkType().toPromise();
  generationHash = await repo.getGenerationHash().toPromise();
  epochAdjustment = await repo.getEpochAdjustment().toPromise();
})();

function clog(signedTx){
    console.log(NODE + "/transactionStatus/" + signedTx.hash);
    console.log(NODE + "/transactions/confirmed/" + signedTx.hash);
    console.log("https://symbol.fyi/transactions/" + signedTx.hash);
    console.log("https://testnet.symbol.fyi/transactions/" + signedTx.hash);
}

<span style="color:red">

## 3.aliceアカウントのリストア </span>

In [3]:
alice = sym.Account.createFromPrivateKey(
    "1E9139CC1580B4AED6A1FE110085281D4982ED0D89CE07F3380EB83069B1****", //🌟ここに3章で作成した秘密鍵を入力
    networkType
);

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229624965-eaf6ae6e-7a61-4cf4-9ca9-1bdfad5cafbf.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229624979-b306ee0b-c83e-4b86-8376-91f031af7856.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229624993-7527cdd1-7919-44b9-9c6f-1f5b3e4d1310.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625002-455e6099-9b95-431d-8686-d09f91b9a16d.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625011-449e35b6-8510-49c6-9f45-e07bce0295dd.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625022-c67dc00f-d002-4120-8a14-fe885a95f670.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625035-d3408a69-bb9a-40cc-b405-70fdf68ccbd2.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625053-4a91f670-e1e1-4d20-a32f-cabb082c5915.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625065-d561e4cd-aaf2-4701-a10e-cd4f7c063cb0.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625087-5411527e-004d-45d6-ba0f-52ccb3c650f7.png">

# 11.制限

アカウントに対する制限とモザイクのグローバル制限についての方法を紹介します。
本章では、既存アカウントの権限を制限してしまうので、使い捨てのアカウントを新規に作成してお試しください。

In [4]:
//使い捨てアカウントCarolの生成
carol = sym.Account.generateNewAccount(networkType);
console.log(carol.address);
//FAUCET URL出力
console.log("https://testnet.symbol.tools/?recipient=" + carol.address.plain() +"&amount=100");

https://testnet.symbol.tools/?recipient=TCWNIRYSAWSKV4K6KFDK3RIZOW67LG4V2QI6WTQ&amount=100

https://testnet.symbol.tools/?recipient=TCWNIRYSAWSKV4K6KFDK3RIZOW67LG4V2QI6WTQ&amount=100